# Creation of the CNN

### Importations 

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
import pandas as pd
import time
import h5py


from torch.autograd import Variable
from torch import nn, optim
from torch.nn import functional as F
from matplotlib import gridspec




print(torch.__version__)

1.0.1.post2


### The Aim 

Some explation about the purpose of this project.

### The Data

Some insight about the data.

In [2]:
Dataset = pd.read_pickle("/home/janjar/Dataset/Trainingset/Training_dataframe.pkl")
Dataset[1029:]

,Room,Array_position,Distance,Absorption,SNR,Audio_file,Phase_Matrix,Label,Angle
1029,1,"[4.39850161295675, 1.5702471407416707, 1.5]",2.0,0.537,[15],/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,30,150.0
1030,1,"[2.0256773003933173, 1.2533760344507048, 1.5]",2.0,0.537,[5],/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,31,155.0
1031,1,"[1.0487413744095253, 3.4388505167580092, 1.5]",2.0,0.537,[15],/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,32,160.0
1032,1,"[2.8673261273287527, 4.995594558899781, 1.5]",2.0,0.537,[10],/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,33,165.0
1033,1,"[4.875989504238883, 3.6932989109940606, 1.5]",2.0,0.537,[20],/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,34,170.0
1034,1,"[4.1969684380281995, 1.3977308096439185, 1.5]",2.0,0.537,[5],/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,35,175.0
1035,1,"[1.8030798618842838, 1.3976947285323391, 1.5]",2.0,0.537,[20],/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,36,180.0


### Usefull variables

#### Defining our number of samples.

In [3]:
nb_freq = 256
nb_time = 360
nb_mics = 4
number_of_rooms = Dataset.shape[0]
print(number_of_rooms)

1036


## Treating the Training Data.

### Loading the phase matrices in tensors with respect to the indexing shown in the dataframe.

In [4]:
#path containing the Training data on the machine you are working on.
#path = "/home/janjar/Dataset/Trainingset/"

data_matrix = np.zeros([number_of_rooms,nb_mics,nb_freq,nb_time])
for i in range(number_of_rooms):
    fileName_matrix = Dataset.iloc[i]['Phase_Matrix']
    #print(fileName_matrix)
    fileObject2 = open(fileName_matrix, 'rb')
    matrix_loaded = pkl.load(fileObject2)
    fileObject2.close()
    data_matrix[i] = matrix_loaded
data_matrix.shape    

(1036, 4, 256, 360)

In [5]:
data_matrix = torch.from_numpy(data_matrix)
data_matrix = data_matrix.view(-1,nb_mics,nb_freq,nb_time)
data_matrix = data_matrix.type('torch.FloatTensor')
data_matrix.shape

torch.Size([1036, 4, 256, 360])

In [6]:
final_data = torch.ones([number_of_rooms*nb_time,nb_mics,nb_freq], dtype=torch.float64)

In [7]:
i = 0
for setup in range(number_of_rooms):
     for nb_tf in range(nb_time):
        final_data[i] = data_matrix[setup,:,:,nb_tf]
        i = i+1
    

In [8]:
final_data.shape

torch.Size([372960, 4, 256])

## Treating the Training Labels.

### Function that reshape a label into the right format for the data_targets.

In [9]:
def prep_targets(index,label):
    target = torch.tensor(label)
    target = target.expand(390,1)
    return target

In [10]:
data_targets = torch.zeros([number_of_rooms], dtype=torch.float64)
for i in range(number_of_rooms):
    data_targets[i] = Dataset.iloc[i]['Label']
data_targets.shape   

torch.Size([1036])

In [11]:
len(data_targets)

1036

In [12]:
final_targets = torch.ones([number_of_rooms*nb_time], dtype=torch.float64)
test = []
for i in range(number_of_rooms):
    curr = [data_targets[i].item()]*nb_time
    test.append(curr)
    

In [13]:
final_targets = torch.FloatTensor(test)
final_targets = final_targets.flatten()
final_targets.shape

torch.Size([372960])

In [14]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [15]:
final_data,final_targets = unison_shuffled_copies(final_data,final_targets)

## Treating the Test Data.

In [16]:
Test_data = h5py.File('DOA_test.hdf5')
X_test = Test_data['features']                   # These are the phase maps for each time frame
Y_test = Test_data['targets']
X_test = np.array(X_test)                        # size = (Number of time frames,1,256,4)
Y_test = np.array(Y_test)
X_test,Y_test = unison_shuffled_copies(X_test,Y_test)
X_test = torch.from_numpy(X_test)
Y_test = torch.from_numpy(Y_test)

X_test.shape, Y_test.shape

(torch.Size([7281, 1, 256, 4]), torch.Size([7281, 37]))

In [17]:
_,b = Y_test[3004].max(dim=0)
b

tensor(30)

# Architecture 1.0 : vanilla CNN for DOA 

 In this section, we will focus on building a vanilla CNN in order to recognize the directions of arrival of the   sound in a specific room. Once done, we will then compare "by hand" the labelel dataset to the predicted values. This is the most basic setup and will try to improve latter.

## 1.1 Creating Models

In [18]:
class Net(nn.Module):
    def __init__(self,nb_hidden = 50, n = 4):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=2)
        self.conv2 = nn.Conv2d(8, 8, kernel_size=2)
        self.conv3 = nn.Conv2d(8 ,8, kernel_size=2)
        self.fc1 = nn.Linear(390096, nb_hidden)# (1x8064) being the dim of the censor obtained by flattening the output of the 3rd CL.
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(nb_hidden*4,nb_hidden)
        self.fc3 = nn.Linear(nb_hidden,2)
        
    
    def forward(self, x, n):
        #print("What actually enters the model:",x.shape)
        test =  x[:,:,0,:,:].view(-1,1,129,390)
        #print("Shape of test:",test.shape)
        output = torch.zeros([390096,1])
        for i in range(n):
            
            y = x[:,:,i,:,:].view(-1,1,129,390)
            y = F.relu(self.conv1(y)) 
            y = F.relu(self.conv2(y))
            y = F.relu(self.conv3(y))
            y = F.relu(self.fc1(y.view(-1, 390096)))
            
            if (i==0):
                output = y
            else:
                output = torch.cat((output,y),1)
                
        output = F.relu(self.fc2(output))
        output = self.dropout(output)
        output = F.softmax(self.fc3(output))        

        return output            

In [19]:
class ws_Net(nn.Module):
    def __init__(self,nb_hidden = 50, n = 390 ):
        super(ws_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=2)
        self.conv2 = nn.Conv2d(8, 8, kernel_size=2)
        self.conv3 = nn.Conv2d(8 ,8, kernel_size=2)
        self.fc1 = nn.Linear(1008, nb_hidden)# (1x1008) being the dim of the censor obtained by flattening the output of the 3rd CL.
        self.fc2 = nn.Linear(nb_hidden*n,512)
        self.fc3 = nn.Linear(512,2)
        
    
    def forward(self, x, n):
        
        #print("What actually enters the model:",x.shape)
        test =  x[:,:,:,:,0].view(-1,1,4,129)
        #print("Shape of test:",test.shape)
        output = torch.zeros([1008,1])
        
        for i in range(n):
            y = x[:,:,:,:,i].view(-1,1,4,129)
            y = F.relu(self.conv1(y)) 
            y = F.relu(self.conv2(y))
            y = F.relu(self.conv3(y))
            y = F.relu(self.fc1(y.view(-1, 1008)))
            
            if (i==0):
                output = y
            else:
                output = torch.cat((output,y),1)
                
        #print("Shape of the out of fc1:",output.shape)        
                
        output = F.relu(self.fc2(output))
        
        #print("Shape of the out of fc2:",output.shape) 

        output = F.softmax(self.fc3(output))   
        
        return output  

In [116]:
class SimpleModel(nn.Module):
    
    def __init__(self, nb_hidden=512,phase_map=64):
        super(SimpleModel, self).__init__()
        self.cl1 = nn.Conv2d(1, phase_map, kernel_size=2)
        self.cl2 = nn.Conv2d(phase_map, phase_map, kernel_size=2)
        self.cl3 = nn.Conv2d(phase_map, phase_map, kernel_size=2)
        #self.dropout = nn.Dropout(0.02)
        self.fc1 = nn.Linear((nb_freq-3)*phase_map,nb_hidden)
        #self.dropout = nn.Dropout(0.02)
        self.fc2 = nn.Linear(nb_hidden,nb_hidden)
        self.fc3 = nn.Linear(nb_hidden,37)
 
 
    def forward(self, x):
       
        x = F.relu(self.cl1(x))
        x = F.relu(self.cl2(x))
        x = F.relu(self.cl3(x))
        #x = self.dropout(x)
        x = F.relu(self.fc1(x.view(-1, (nb_freq-3)*64)))
        #x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

    def compute_nb_errors(self,model, data_input, data_target, mini_batch_size):

        nb_data_errors = 0
        for b in range(0, data_input.size(0), mini_batch_size):
            a = model(data_input.narrow(0, b, mini_batch_size))
            val = torch.max(a,1)[1]
            for k in range(mini_batch_size):
                if data_target.data[b + k] != val[k]:
                    nb_data_errors = nb_data_errors + 1

        return nb_data_errors

### Preparing the Data

In [66]:
def prep_input_vanilla(train_input):
    new_train_input = train_input.view(-1,1,nb_mics,nb_freq)
    return new_train_input

In [67]:
def prep_input_model2(train_input):
    new_train_input = train_input.view(-1,1,nb_mics,nb_freq,nb_time)
    return new_train_input

In [68]:
train_data = prep_input_vanilla(final_data)
train_data = train_data.float()
train_data.shape

torch.Size([372960, 1, 4, 256])

In [69]:
train_data2 = prep_input_model2(data_matrix)
train_data2.shape

torch.Size([1036, 1, 4, 256, 360])

In [70]:
test_data = prep_input_vanilla(X_test[0:7280])
test_data = test_data.float()
test_data.shape

torch.Size([7280, 1, 4, 256])

### Turning test labels from Hot-One to Index

In [71]:
_,test_targets = Y_test[0:7280].max(dim=1)
test_targets[3004]

tensor(30)

train_data = prep_input_model2(data_matrix)
train_data.shape

In [72]:
def prep_labels_vanilla(train_input):
    new_train_input = train_input.view(-1)
    return new_train_input

In [73]:
train_targets = prep_labels_vanilla(final_targets)
train_targets = train_targets.to(dtype=torch.int64)
train_targets2 =  prep_labels_vanilla(data_targets)
train_data, train_targets, train_data2, train_targets2 = Variable(train_data), Variable(train_targets), Variable(train_data2),Variable(train_targets2)
train_data.shape,train_data2.shape,train_targets.shape,train_targets2.shape

(torch.Size([372960, 1, 4, 256]),
 torch.Size([1036, 1, 4, 256, 360]),
 torch.Size([372960]),
 torch.Size([1036]))

### Spliting the Training Data into Training/Validation.

In [74]:
training_data = train_data[0:300000]
training_targets = train_targets[0:300000]
validation_data = train_data[10000:]
validation_targets = train_targets[10000:]
training_data.shape, validation_data.shape, training_targets.shape, validation_targets.shape,len(training_targets),len(validation_targets)

(torch.Size([300000, 1, 4, 256]),
 torch.Size([362960, 1, 4, 256]),
 torch.Size([300000]),
 torch.Size([362960]),
 300000,
 362960)

### End of Sanity check.

## 1.2 Training the model

In [126]:
model = Net()

In [127]:
model1 = ws_Net()

In [128]:
model2 = (SimpleModel()).float()

In [129]:
pytorch_total_params = sum(p.numel() for p in model2.parameters() if p.requires_grad)
pytorch_total_params

8605669

In [130]:
mini_batch_size = 1
nb_epochs = 100
eta = 0.001 #learning rate
criterion = torch.nn.CrossEntropyLoss() #Cross Entropy
optimizer = torch.optim.Adam(model2.parameters(), lr = eta, weight_decay=0.001) #ADAM
optim_no_wd = torch.optim.Adam(model2.parameters(), lr = eta)
net_ch = 4
ws_ch = 390

### Need to figure out how to use mini-batches to gain time in this configuration.
#### train_model is the method used to update the weights.

In [131]:
def train_model_3(model, optimizer1, nb_epochs, train_input, train_target ,mini_batch_size,n):

    start = time.time()
    for e in range(0,nb_epochs):
        start_ep = time.time()
        for b in range(0, train_input.size(0), mini_batch_size):
            start = time.time()
            #print("Shape of the input of the model:",train_input.narrow(0, b, mini_batch_size).shape)
            output = model(train_input.narrow(0, b, mini_batch_size),n)
            target = train_target.narrow(0, b, mini_batch_size)
            _,indices = output.max(1)
            #print("batch :",b," Output :",output,"Targets:", target)
            loss = criterion(output,target.long())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        if (e == 0):
            end_ep = time.time()
            print("Time for 1 epochs is :{:5}".format(-(start_ep-end_ep)))
            
        print("Loss for epoch{:3} is {:5} ".format(e,loss.data.item()))
            
    end = time.time()
    print("Time the hole training is :{:5}".format(-(start-end)))

       

    

In [132]:
def train_simple_model(model, optimizer, nb_epochs, train_input, train_target ,mini_batch_size):

    start = time.time()
    for e in range(0,nb_epochs):
        start_ep = time.time()
        for b in range(0, train_input.size(0), mini_batch_size):
            start = time.time()
            #print("Shape of the input of the model:",train_input.narrow(0, b, mini_batch_size).shape)
            output = model((train_input.narrow(0, b, mini_batch_size)).float())
            target = train_target.narrow(0, b, mini_batch_size)
            _,indices = output.max(1)
            print("batch :",b," Output :",output,"Targets:", target)
            loss = criterion(output,target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        if (e == 0):
            end_ep = time.time()
            print("Time for 1 epochs is :{:5}".format(-(start_ep-end_ep)))
            
        print("Loss for epoch{:3} is {:5} ".format(e,loss.data.item()))
            
    end = time.time()
    print("Time the hole training is :{:5}".format(-(start-end)))

       

In [ ]:
train_simple_model(model2,optimizer,nb_epochs,test_data[:10],test_targets[:10],mini_batch_size)

/home/janjar/test/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


batch : 0  Output : tensor([[0.0265, 0.0263, 0.0265, 0.0278, 0.0274, 0.0261, 0.0286, 0.0271, 0.0275,
         0.0271, 0.0259, 0.0258, 0.0262, 0.0279, 0.0263, 0.0261, 0.0285, 0.0277,
         0.0270, 0.0264, 0.0268, 0.0273, 0.0265, 0.0288, 0.0263, 0.0268, 0.0282,
         0.0264, 0.0270, 0.0273, 0.0272, 0.0278, 0.0270, 0.0264, 0.0274, 0.0276,
         0.0265]], grad_fn=<SoftmaxBackward>) Targets: tensor([18])
batch : 1  Output : tensor([[0.0279, 0.0237, 0.0245, 0.0240, 0.0252, 0.0213, 0.0303, 0.0236, 0.0237,
         0.0293, 0.0258, 0.0246, 0.0260, 0.0279, 0.0284, 0.0241, 0.0307, 0.0286,
         0.0522, 0.0234, 0.0272, 0.0290, 0.0225, 0.0301, 0.0276, 0.0287, 0.0254,
         0.0254, 0.0264, 0.0279, 0.0287, 0.0274, 0.0244, 0.0266, 0.0252, 0.0279,
         0.0245]], grad_fn=<SoftmaxBackward>) Targets: tensor([24])
batch : 2  Output : tensor([[0.0263, 0.0170, 0.0191, 0.0173, 0.0211, 0.0154, 0.0283, 0.0199, 0.0203,
         0.0272, 0.0233, 0.0212, 0.0250, 0.0263, 0.0324, 0.0167, 0.0310, 0.

batch : 9  Output : tensor([[1.1801e-06, 1.4367e-07, 5.1736e-07, 7.9617e-07, 5.1044e-07, 1.9623e-07,
         1.2015e-03, 2.4694e-07, 9.1677e-07, 2.1654e-06, 7.2442e-07, 2.9453e-07,
         3.9418e-04, 1.1626e-06, 3.1505e-06, 3.2133e-03, 2.7793e-06, 4.0083e-07,
         9.8252e-01, 5.0872e-07, 2.4311e-06, 5.9599e-07, 3.2406e-07, 5.9043e-07,
         3.6347e-03, 1.9085e-07, 6.2474e-07, 9.0075e-03, 2.2251e-07, 1.0147e-06,
         1.4635e-06, 4.0340e-07, 4.8884e-07, 6.5448e-07, 2.4973e-07, 1.1216e-06,
         5.3003e-07]], grad_fn=<SoftmaxBackward>) Targets: tensor([27])
Loss for epoch  1 is 3.646540403366089 
batch : 0  Output : tensor([[6.7413e-11, 3.1184e-12, 1.7731e-11, 3.7626e-11, 1.0904e-11, 2.5154e-12,
         4.0246e-06, 4.7721e-12, 2.9113e-11, 1.6266e-10, 3.5701e-11, 5.7338e-12,
         4.3610e-07, 7.9495e-11, 2.7554e-10, 1.7010e-05, 2.1291e-10, 1.3236e-11,
         9.9994e-01, 1.0526e-11, 1.5773e-10, 2.2030e-11, 7.2481e-12, 2.3097e-11,
         2.0505e-05, 4.4597e-12, 1.697

batch : 8  Output : tensor([[8.4850e-14, 4.2099e-15, 1.8521e-14, 2.0183e-13, 8.3551e-14, 5.3843e-14,
         5.6978e-10, 1.4227e-13, 1.3846e-13, 3.8336e-13, 2.5041e-14, 3.5876e-13,
         1.1497e-09, 1.4643e-13, 6.7595e-13, 9.8758e-09, 1.3098e-12, 1.8239e-14,
         7.9240e-09, 1.2942e-13, 4.5282e-13, 1.1418e-13, 1.6708e-13, 5.7659e-14,
         4.8091e-09, 4.6428e-14, 4.7835e-13, 1.0000e+00, 2.6809e-14, 1.2017e-13,
         6.2437e-14, 2.8547e-14, 6.4270e-14, 3.0269e-14, 1.4099e-14, 4.7426e-13,
         3.8733e-14]], grad_fn=<SoftmaxBackward>) Targets: tensor([27])
batch : 9  Output : tensor([[7.9473e-12, 6.1199e-13, 2.1361e-12, 1.7552e-11, 8.3054e-12, 5.3970e-12,
         1.5145e-08, 1.3522e-11, 1.1274e-11, 3.0781e-11, 3.0452e-12, 3.0630e-11,
         2.8105e-08, 1.4616e-11, 4.5392e-11, 2.0529e-07, 8.4803e-11, 2.0939e-12,
         2.0841e-07, 1.1942e-11, 3.0985e-11, 1.0916e-11, 1.4863e-11, 5.9759e-12,
         9.6592e-08, 5.3172e-12, 3.4766e-11, 1.0000e+00, 3.0596e-12, 1.0040e-1

batch : 7  Output : tensor([[3.5075e-23, 1.9782e-23, 9.4524e-24, 1.0549e-24, 1.4443e-24, 1.4129e-25,
         2.1078e-14, 2.2141e-25, 9.1122e-25, 1.1253e-23, 2.4520e-23, 1.5552e-25,
         1.8905e-16, 7.8087e-24, 8.8737e-24, 2.9460e-12, 5.8458e-24, 2.1676e-24,
         1.0000e+00, 1.4413e-25, 1.4979e-24, 6.9761e-25, 1.5358e-25, 6.9948e-24,
         6.1596e-15, 3.8429e-24, 2.3479e-25, 3.0287e-23, 1.4190e-24, 1.2562e-23,
         4.8415e-24, 4.9250e-24, 5.2175e-25, 1.2787e-23, 1.5298e-23, 7.0899e-26,
         1.0221e-24]], grad_fn=<SoftmaxBackward>) Targets: tensor([18])
batch : 8  Output : tensor([[1.5551e-06, 1.5880e-06, 1.3253e-06, 8.1078e-07, 9.2910e-07, 4.0814e-07,
         5.6909e-04, 4.8903e-07, 6.4132e-07, 1.7277e-06, 1.5636e-06, 4.1760e-07,
         1.5793e-04, 1.1128e-06, 1.3909e-06, 4.3355e-03, 8.6413e-07, 1.0151e-06,
         9.9456e-01, 4.4475e-07, 6.5444e-07, 6.5756e-07, 4.4256e-07, 1.1534e-06,
         3.4908e-04, 9.7650e-07, 5.4623e-07, 1.0255e-06, 8.6833e-07, 1.3953e-0

...

batch : 8  Output : tensor([[3.2767e-09, 4.1595e-09, 3.8971e-09, 3.4417e-09, 3.7884e-09, 4.1633e-09,
         3.4358e-09, 3.9145e-09, 3.5156e-09, 2.8319e-09, 3.6503e-09, 3.8639e-09,
         1.7955e-11, 3.3608e-09, 2.6410e-09, 2.8278e-09, 2.7230e-09, 3.9641e-09,
         1.0000e+00, 3.7572e-09, 3.2671e-09, 3.7248e-09, 3.8473e-09, 3.5673e-09,
         1.0737e-08, 3.9914e-09, 3.5522e-09, 6.6046e-12, 3.8738e-09, 3.4357e-09,
         3.3469e-09, 3.5831e-09, 3.7842e-09, 3.8002e-09, 3.9232e-09, 3.4372e-09,
         3.8090e-09]], grad_fn=<SoftmaxBackward>) Targets: tensor([27])
batch : 9  Output : tensor([[5.4157e-10, 7.0128e-10, 6.5360e-10, 5.6964e-10, 6.3370e-10, 7.0205e-10,
         4.5563e-10, 6.5659e-10, 5.8417e-10, 4.6073e-10, 6.0887e-10, 6.4689e-10,
         2.1845e-12, 5.5548e-10, 4.2787e-10, 4.1081e-10, 4.4243e-10, 6.6570e-10,
         1.0000e+00, 6.2792e-10, 5.3953e-10, 6.2190e-10, 6.4444e-10, 5.9319e-10,
         2.1990e-09, 6.7056e-10, 5.9004e-10, 6.7895e-13, 6.4930e-10, 5.7005e-1

batch : 5  Output : tensor([[2.1220e-09, 2.6611e-09, 2.5075e-09, 2.2287e-09, 2.4437e-09, 2.6616e-09,
         1.8824e-09, 2.5158e-09, 2.2757e-09, 1.8482e-09, 2.3603e-09, 2.4853e-09,
         9.7767e-12, 2.1757e-09, 1.7262e-09, 1.6645e-09, 1.7776e-09, 2.5478e-09,
         1.0000e+00, 2.4219e-09, 2.1175e-09, 2.4022e-09, 2.4761e-09, 2.3072e-09,
         8.0305e-09, 2.5648e-09, 2.2977e-09, 3.5192e-12, 2.4946e-09, 2.2254e-09,
         2.1695e-09, 2.3176e-09, 2.4384e-09, 2.4483e-09, 2.5241e-09, 2.2261e-09,
         2.4547e-09]], grad_fn=<SoftmaxBackward>) Targets: tensor([18])
batch : 6  Output : tensor([[6.2160e-07, 7.3101e-07, 7.0021e-07, 6.4295e-07, 6.8708e-07, 7.3073e-07,
         5.6167e-07, 7.0168e-07, 6.5306e-07, 5.6069e-07, 6.7040e-07, 6.9512e-07,
         1.5730e-08, 6.3274e-07, 5.3383e-07, 4.6956e-07, 5.4580e-07, 7.0830e-07,
         9.9998e-01, 6.8261e-07, 6.1976e-07, 6.7864e-07, 6.9361e-07, 6.6005e-07,
         1.4272e-06, 7.1145e-07, 6.5708e-07, 7.1284e-09, 6.9764e-07, 6.4244e-0

batch : 3  Output : tensor([[1.4718e-11, 1.9226e-11, 1.7959e-11, 1.5868e-11, 1.7468e-11, 1.9189e-11,
         4.5587e-11, 1.8022e-11, 1.6106e-11, 1.2625e-11, 1.6769e-11, 1.7857e-11,
         6.8221e-15, 1.5326e-11, 1.1661e-11, 1.4370e-11, 1.1974e-11, 1.8303e-11,
         1.0000e+00, 1.7285e-11, 1.4784e-11, 1.7099e-11, 1.7715e-11, 1.6417e-11,
         4.0599e-11, 1.8468e-11, 1.6371e-11, 3.0143e-15, 1.7887e-11, 1.5663e-11,
         1.5127e-11, 1.6478e-11, 1.7380e-11, 1.7516e-11, 1.8117e-11, 1.5734e-11,
         1.7547e-11]], grad_fn=<SoftmaxBackward>) Targets: tensor([6])
batch : 4  Output : tensor([[2.9680e-08, 3.5535e-08, 3.3923e-08, 3.0781e-08, 3.3229e-08, 3.5508e-08,
         2.0303e-08, 3.3983e-08, 3.1398e-08, 2.6361e-08, 3.2353e-08, 3.3608e-08,
         4.3844e-10, 3.0249e-08, 2.4883e-08, 1.8972e-08, 2.5548e-08, 3.4353e-08,
         1.0000e+00, 3.2963e-08, 2.9531e-08, 3.2753e-08, 3.3551e-08, 3.1764e-08,
         8.5203e-08, 3.4523e-08, 3.1589e-08, 1.3187e-10, 3.3790e-08, 3.0801e-08

batch : 9  Output : tensor([[3.8550e-08, 4.5549e-08, 4.3696e-08, 4.0066e-08, 4.2905e-08, 4.5507e-08,
         2.9964e-08, 4.3759e-08, 4.0715e-08, 3.4655e-08, 4.1877e-08, 4.3345e-08,
         4.4081e-10, 3.9356e-08, 3.2792e-08, 2.6624e-08, 3.3534e-08, 4.4202e-08,
         1.0000e+00, 4.2571e-08, 3.8450e-08, 4.2317e-08, 4.3261e-08, 4.1181e-08,
         1.0669e-07, 4.4418e-08, 4.0994e-08, 1.6854e-10, 4.3561e-08, 3.9995e-08,
         3.9198e-08, 4.1296e-08, 4.2758e-08, 4.2957e-08, 4.3931e-08, 3.9965e-08,
         4.3014e-08]], grad_fn=<SoftmaxBackward>) Targets: tensor([27])
Loss for epoch 83 is 3.6563117504119873 
batch : 0  Output : tensor([[1.1067e-09, 1.3507e-09, 1.2845e-09, 1.1558e-09, 1.2555e-09, 1.3486e-09,
         7.7587e-10, 1.2862e-09, 1.1797e-09, 9.6869e-10, 1.2199e-09, 1.2715e-09,
         6.6535e-12, 1.1328e-09, 9.0597e-10, 6.2218e-10, 9.3317e-10, 1.3027e-09,
         1.0000e+00, 1.2439e-09, 1.0994e-09, 1.2360e-09, 1.2683e-09, 1.1964e-09,
         2.9105e-09, 1.3094e-09, 1.18

batch : 8  Output : tensor([[6.2993e-07, 7.1730e-07, 6.9517e-07, 6.5123e-07, 6.8586e-07, 7.1697e-07,
         5.6806e-07, 6.9608e-07, 6.5874e-07, 5.8067e-07, 6.7319e-07, 6.9114e-07,
         1.3214e-08, 6.4160e-07, 5.5487e-07, 4.7103e-07, 5.6459e-07, 7.0141e-07,
         9.9998e-01, 6.8156e-07, 6.2921e-07, 6.7827e-07, 6.8992e-07, 6.6476e-07,
         1.3432e-06, 7.0413e-07, 6.6267e-07, 6.1571e-09, 6.9382e-07, 6.4903e-07,
         6.3857e-07, 6.6596e-07, 6.8362e-07, 6.8625e-07, 6.9811e-07, 6.4931e-07,
         6.8713e-07]], grad_fn=<SoftmaxBackward>) Targets: tensor([27])
batch : 9  Output : tensor([[1.7670e-07, 2.0334e-07, 1.9658e-07, 1.8294e-07, 1.9372e-07, 2.0327e-07,
         1.3528e-07, 1.9686e-07, 1.8542e-07, 1.6157e-07, 1.8986e-07, 1.9527e-07,
         2.9512e-09, 1.8008e-07, 1.5388e-07, 1.1990e-07, 1.5686e-07, 1.9847e-07,
         9.9999e-01, 1.9241e-07, 1.7642e-07, 1.9137e-07, 1.9497e-07, 1.8719e-07,
         4.3203e-07, 1.9930e-07, 1.8652e-07, 1.2182e-09, 1.9616e-07, 1.8248e-0

In [228]:
test_targets[0:10]

tensor([24, 18,  6, 24, 30, 30, 24, 12,  6,  6])

In [229]:
test = model2(test_data[0:10])
_,pred = test.max(1)
pred

/home/janjar/test/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([12, 12, 12, 12, 12, 12, 12, 12, 12, 12])

In [ ]:
def train_simple_model(model, optimizer, nb_epochs, train_input, train_target ,mini_batch_size):

    start = time.time()
    for e in range(0,nb_epochs):
        start_ep = time.time()
        for b in range(0, train_input.size(0), mini_batch_size):
            start = time.time()
            #print("Shape of the input of the model:",train_input.narrow(0, b, mini_batch_size).shape)
            output = model((train_input.narrow(0, b, mini_batch_size)).float())
            target = train_target.narrow(0, b, mini_batch_size)
            _,indices = output.max(1)
            #print("batch :",b," Output :",indices,"Targets:", target)
            loss = criterion(output,target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        if (e == 0):
            end_ep = time.time()
            print("Time for 1 epochs is :{:5}".format(-(start_ep-end_ep)))
            
        print("Loss for epoch{:3} is {:5} ".format(e,loss.data.item()))
            
    end = time.time()
    print("Time the hole training is :{:5}".format(-(start-end)))

